$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{University of Los Andes}$$
$$\textrm{High Energy Physics Group: Phenomenology of Particles}$$

This code was written to be running in Docker. If you do not have a Docker inside hep-server2 please refer to: https://github.com/Phenomenology-group-uniandes/Tutoriales_Generales

In [1]:
import os, sys

def add_parent_lib_path(name="Semanas_07_08_09"):
    sys.path.append(os.path.join(sys.path[0].split(name)[0],name))
    
add_parent_lib_path()

In [2]:
from Uniandes_Framework.ml_tools import tools
from Uniandes_Framework.delphes_reader import root_analysis

import pandas as pd

from ROOT import TCanvas #Permite poner los histogramas
from ROOT import THStack #Permite graficar varios histogramas al mismo tiempo
from ROOT import TLegend #Permite poner legends cuando se sobrelapan histogramas
from ROOT import TLatex #Permite poner avisos en Latex en las graficas

Welcome to JupyROOT 6.22/06


In [3]:
Path_Leptoquarks_searches = os.path.dirname(os.path.dirname(os.path.realpath('XBG_Discriminator_Histograms.ipynb')))

In [4]:
channels = {"hadronic_dLQ": 'b_b_tau_tau_hadronic', 
            "hadronic_sLQ": 'b_tau_tau_hadronic', 
            "semileptonic_dLQ": 'b_b_tau_tau_semileptonic', 
            "semileptonic_sLQ": 'b_tau_tau_semileptonic'}

Masses = ['1250', '1500', '1750', '2000', '2250', '2500']
bkgs = ['ww', 'wz', 'zz']

In [5]:
def Read_Cutflows_N_Events(channel, signal, Luminosity = 137*1000):
    
    #[Luminosity]: 1/pb

    cutflows_path = os.path.join(Path_Leptoquarks_searches, '03_delphes_preselection/Cutflows', f'{channel}.csv')
    cutflows_file = pd.read_csv(cutflows_path, index_col= 0)
    cutflows_file = cutflows_file.dropna()
    
    Efficience = cutflows_file[signal]['DeltaR > 0.3']/cutflows_file[signal]['All']
    XS = cutflows_file[signal]['XS']
    return Efficience*XS*Luminosity

In [6]:
histograms = {}

for channel in channels:    
    histograms[channel] = {}
    for Mass in Masses:
                
        path_root_file = f'/disco4/personal_folders/Joaquin/2023/Semanas_07_08_09/Leptoquarks_Searches_2022/05_refined_ML_output/MLQ_{Mass}/{channels[channel]}/Gradient_Boosting.root'       
        histograms[channel][f'M{Mass}'] = root_analysis.Read_ROOT_File(path_root_file, expected_keys = [f'{bkg}_{channels[channel]}' for bkg in bkgs])

In [7]:
# canvas = TCanvas('','',0,0,800,600)

In [8]:
# histograms['hadronic_dLQ']['M1250']['ww_b_b_tau_tau_hadronic'].Draw('hist')
# canvas.Draw()

In [9]:
for channel in channels:    
    for Mass in Masses:   
        for signal_key in histograms[channel][f'M{Mass}'].keys():
            N_events = Read_Cutflows_N_Events(channel, signal_key.replace(f'_{channels[channel]}',''))
            histograms[channel][f'M{Mass}'][signal_key].Scale(N_events/histograms[channel][f'M{Mass}'][signal_key].Integral())

In [10]:
# histograms['hadronic_dLQ']['M1250']['ww_b_b_tau_tau_hadronic'].Draw('hist')
# canvas.Draw()

In [11]:
# histograms['hadronic_dLQ']['M1250']

In [12]:
histograms_list = {}
for channel in channels:    
    histograms_list[channel] = {}
    for Mass in Masses:
        histograms_list[channel][f'M{Mass}'] = [histograms[channel][f'M{Mass}'][signal_key] for signal_key in histograms[channel][f'M{Mass}'].keys()]

In [13]:
# histograms_list

In [14]:
!rm -rf Histograms_Diboson/
!mkdir Histograms_Diboson/

In [15]:
histograms_Diboson = {}
for channel in channels:   
    try: os.mkdir(f'Histograms_Diboson/{channel}')
    except: pass

    histograms_Diboson[channel] = {}
    for Mass in Masses:
        histograms_Diboson[channel][f'M{Mass}'] = root_analysis.sum_histos(histograms_list[channel][f'M{Mass}'])
        histograms_Diboson[channel][f'M{Mass}'].Scale(1.0/histograms_Diboson[channel][f'M{Mass}'].Integral())
        histograms_Diboson[channel][f'M{Mass}'].SetNameTitle(f'Diboson_M{Mass}', f'Diboson_M{Mass}')
        
    root_analysis.Write_ROOT_File(f'Histograms_Diboson/{channel}/Diboson.root', histograms_Diboson[channel])
    root_analysis.Write_txt_file_with_high_per_bin(f'Histograms_Diboson/{channel}/Diboson', histograms_Diboson[channel])
    root_analysis.Save_Histograms_png(f'Histograms_Diboson/{channel}', histograms_Diboson[channel], Log_Y= True )

Info in <TCanvas::Print>: png file Histograms_Diboson/hadronic_dLQ/histograms_M1250.png has been created
Info in <TCanvas::Print>: png file Histograms_Diboson/hadronic_dLQ/histograms_M1500.png has been created
Info in <TCanvas::Print>: png file Histograms_Diboson/hadronic_dLQ/histograms_M1750.png has been created
Info in <TCanvas::Print>: png file Histograms_Diboson/hadronic_dLQ/histograms_M2000.png has been created
Info in <TCanvas::Print>: png file Histograms_Diboson/hadronic_dLQ/histograms_M2250.png has been created
Info in <TCanvas::Print>: png file Histograms_Diboson/hadronic_dLQ/histograms_M2500.png has been created
Info in <TCanvas::Print>: png file Histograms_Diboson/hadronic_sLQ/histograms_M1250.png has been created
Info in <TCanvas::Print>: png file Histograms_Diboson/hadronic_sLQ/histograms_M1500.png has been created
Info in <TCanvas::Print>: png file Histograms_Diboson/hadronic_sLQ/histograms_M1750.png has been created
Info in <TCanvas::Print>: png file Histograms_Diboson/h